In [23]:
import nbformat as nbf
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import ast
import re

In [20]:
template = '/home/fbrito/dcs-s2-ndvi-boa/src/main/app-resources/application_template.xml'

descriptor_file = '/home/fbrito/dcs-s2-ndvi-boa/src/main/app-resources/application.xml'

nb_source = '/home/fbrito/dcs-s2-ndvi-boa/src/main/app-resources/notebook/libexec/input.ipynb'

In [43]:


nb = nbf.read(nb_source, 4)

tree = etree.parse(template)
root = tree.getroot()

for index, cell in enumerate(nb['cells']):
    print index
    
    if str(cell['cell_type']) == 'code': 
        
        try:
            root_ast = ast.parse(str(cell['source']))
            names = list({node.id for node in ast.walk(root_ast) if isinstance(node, ast.Name)})
           
            if len(names) == 1:
            
                if names[0] == 'input_reference': #and action == 'runtime':
                    print('a')
                    el_source = root.xpath('/application/workflow/node[@id="notebook"]/sources/source',
                                            namespaces={}) 
        
                    print(el_source)
                    el_source[0].text = re.findall('"([^\']*)"', str(nb['cells'][7]['source']))[0]
            
                    print(str(cell['source']))
            
            if len(names) != 2:
                continue 
                                
            if names[0] == 'dict' or names[1] == 'dict':
                print('here')
                # deal with the alphabetical order
                if names[1] == 'dict': 
                    names[1] = names[0]
                    names[0] = 'dict'
                    
                print(str(cell['source']))   
                exec(str(cell['source']))
                
                if 'title' in eval(names[1]).keys() and 'abstract' in eval(names[1]).keys() and 'id' in eval(names[1]).keys():
                    
                    if 'value' in eval(names[1]).keys():
                        # it's a parameter
                        xml_string = '<parameter id="%s" title="%s" abstract="%s" maxOccurs="1" scope="runtime" type="LiteralData">%s</parameter>' % (eval(names[1])['id'], 
                                  eval(names[1])['title'], 
                                  eval(names[1])['abstract'], 
                                  eval(names[1])['value'])
                        
                        el_default_params = root.xpath('/application/jobTemplates/jobTemplate/defaultParameters', 
                                namespaces={})
                        
                        el_default_params[0].append(etree.fromstring(xml_string))
                     
                    else:
                       
                        # it's the service definition
                        el_workflow = root.xpath('/application/workflow',
                                            namespaces={}) 
        
                        el_workflow[0].attrib['id'] = eval(names[1])['id']
                        el_workflow[0].attrib['title'] = eval(names[1])['title']
                        el_workflow[0].attrib['abstract'] = eval(names[1])['abstract']
                    
        except SyntaxError:
            continue
          
descriptor = open(descriptor_file, 'wb')
descriptor.write(etree.tostring(tree, pretty_print=True))
descriptor.close()

0
1
2
here
pa_code = dict([('id', 'pa_code'),
               ('value', 'DO'),
               ('title', 'Protected Area code'),
               ('abstract', 'Protected Area code (one of DO,...)')])
3
here
service = dict([('title', 'NDVI BOA - updated 2'),
                ('abstract', 'Sentinel-2 NDVI BOA'),
                ('id', 'ndvi_boa')])
4
here
pa_name = dict([('id', 'pa_name'),
               ('value', 'Donana'),
               ('title', 'Protected Area name'),
               ('abstract', 'Protected Area name (one of Donana,...)')])
5
6
7
a
[<Element source at 0x7fb4fd2f59e0>]
input_reference = "https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20170909T110651_N0205_R137_T29SPB_20170909T111217"
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [48]:
nb['cells'][7]

{u'cell_type': u'code',
 u'execution_count': 10,
 u'metadata': {u'collapsed': True},
 u'outputs': [],
 u'source': u"input_identifier = 'S2A_MSIL2A_20170909T110651_N0205_R137_T29SPB_20170909T111217'"}

In [20]:
exec(str(nb['cells'][5]['source']))

In [22]:
type(a_par)

dict

In [23]:
type(pa_name)

dict

In [44]:
root_ast = ast.parse(str(nb['cells'][2]['source']))
names = list({node.id for node in ast.walk(root_ast) if isinstance(node, ast.Name)})
            

In [45]:
names

['dict', 'pa_code']

In [30]:
root_ast = ast.parse(str(nb['cells'][4]['source']))
names = {node.id for node in ast.walk(root_ast) if isinstance(node, ast.Name)}

In [31]:
names

{'dict', 'pa_name'}

In [49]:
len(names)

1

In [13]:
root.xpath('/application/workflow/node[@id="notebook"]/sources/source',
                                            namespaces={}) 

[<Element source at 0x7fb4fd6475a8>]

In [5]:
el_source

[]

In [39]:
re.findall('"([^\']*)"', str(nb['cells'][7]['source']))[0]

'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20170909T110651_N0205_R137_T29SPB_20170909T111217'

In [31]:
str(nb['cells'][7]['source'])

'input_reference = "https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20170909T110651_N0205_R137_T29SPB_20170909T111217"'